In [1]:
from IPython.display import HTML, display
from copy import deepcopy
import random
import os
import openai
import time
import sys
import ast
import json

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from getpass import getpass
import os
import pandas as pd

import pdfkit
import html
from copy import deepcopy
import re

In [2]:
# setting up an OpenAI template on the run
OPENAI_API_KEY = getpass()

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

········


### Setup Model

In [3]:
model = ChatOpenAI(model="gpt-4-0613")
model.temperature = 0.8

### Multi-shot Example

In [20]:
example_1 = [ { "from": "human",  "value": "Interact with a household to solve a task. Imagine you are an intelligent agent in a household environment and your target is to perform actions to complete the task goal. At the beginning of your interactions, you will be given the detailed description of the current environment and your goal to accomplish. For each of your turn, you will be given a list of actions which you can choose one to perform in this turn. You should choose from two actions: \"THOUGHT\" or \"ACTION\". If you choose \"THOUGHT\", you should first think about the current condition and plan for your future actions, and then output your action in this turn. Your output must strictly follow this format:\"THOUGHT: your thoughts.\n ACTION: your next action\n\"; If you choose \"ACTION\", you should directly output the action in this turn. Your output must strictly follow this format:\"ACTION: your next action\n\". After your each turn, the environment will give you immediate feedback based on which you plan your next few steps. if the envrionment output \"Nothing happened\", that means the previous action is invalid and you should try more options.\n Reminder: \n1. the action must be chosen from the given available actions. Any actions except provided available actions will be regarded as illegal. \n2. Think when necessary, try to act directly more in the process." }, { "from": "gpt", "value": "OK. I'll follow your instructions and try my best to solve the task." }, { "from": "human",  "value": "Here is your task. You are in the middle of a room. Looking quickly around you, you see a bed 1, a desk 1, a drawer 8, a drawer 7, a drawer 6, a drawer 5, a drawer 4, a drawer 3, a drawer 2, a drawer 1, a dresser 1, a garbagecan 1, a shelf 5, a shelf 4, a shelf 3, a shelf 2, and a shelf 1.\nYour task is to: put two cellphone in dresser." }, { "from": "gpt",  "value": "THOUGHT: First, I need to find the cellphones. I'm going to start by looking in the most likely locations, such as the desk and the drawers.\n ACTION: go to desk 1" }, { "from": "human",  "value": "On the desk 1, you see a book 1, a bowl 1, a keychain 1, a laptop 1, and a pencil 2." }, { "from": "gpt",  "value": "ACTION: go to drawer 1" }, { "from": "human",  "value": "The drawer 1 is closed." }, { "from": "gpt",  "value": "ACTION: open drawer 1" }, { "from": "human",  "value": "You open the drawer 1. The drawer 1 is open. In it, you see nothing." }, { "from": "gpt",  "value": "ACTION: go to drawer 2" }, { "from": "human",  "value": "The drawer 2 is closed." }, { "from": "gpt",  "value": "ACTION: open drawer 2" }, { "from": "human",  "value": "You open the drawer 2. The drawer 2 is open. In it, you see nothing." }, { "from": "gpt",  "value": "ACTION: go to drawer 3" }, { "from": "human",  "value": "The drawer 3 is closed." }, { "from": "gpt",  "value": "ACTION: open drawer 3" }, { "from": "human",  "value": "You open the drawer 3. The drawer 3 is open. In it, you see nothing." }, { "from": "gpt",  "value": "ACTION: go to drawer 4" }, { "from": "human",  "value": "The drawer 4 is closed." }, { "from": "gpt",  "value": "ACTION: open drawer 4" }, { "from": "human",  "value": "You open the drawer 4. The drawer 4 is open. In it, you see nothing." }, { "from": "gpt",  "value": "ACTION: go to drawer 5" }, { "from": "human",  "value": "The drawer 5 is closed." }, { "from": "gpt",  "value": "ACTION: open drawer 5" }, { "from": "human",  "value": "You open the drawer 5. The drawer 5 is open. In it, you see nothing." }, { "from": "gpt",  "value": "ACTION: go to drawer 6" }, { "from": "human",  "value": "The drawer 6 is closed." }, { "from": "gpt",  "value": "ACTION: open drawer 6" }, { "from": "human",  "value": "You open the drawer 6. The drawer 6 is open. In it, you see nothing." }, { "from": "gpt",  "value": "ACTION: go to drawer 7" }, { "from": "human",  "value": "On the drawer 7, you see nothing." }, { "from": "gpt",  "value": "ACTION: open drawer 7" }, { "from": "human",  "value": "On the drawer 7, you see nothing." }, { "from": "gpt",  "value": "ACTION: go to drawer 8" }, { "from": "human",  "value": "The drawer 8 is closed." }, { "from": "gpt",  "value": "ACTION: open drawer 8" }, { "from": "human",  "value": "You open the drawer 8. The drawer 8 is open. In it, you see a creditcard 3." }, { "from": "gpt",  "value": "ACTION: go to shelf 1" }, { "from": "human",  "value": "On the shelf 1, you see nothing." }, { "from": "gpt",  "value": "ACTION: go to shelf 2" }, { "from": "human",  "value": "On the shelf 2, you see a alarmclock 1, a cellphone 1, and a creditcard 1." }, { "from": "gpt",  "value": "ACTION: take cellphone 1 from shelf 2" }, { "from": "human",  "value": "You pick up the cellphone 1 from the shelf 2." }, { "from": "gpt",  "value": "ACTION: go to shelf 3" }, { "from": "human",  "value": "On the shelf 3, you see a pencil 1." }, { "from": "gpt",  "value": "ACTION: go to shelf 4" }, { "from": "human",  "value": "On the shelf 4, you see a cd 1." }, { "from": "gpt",  "value": "ACTION: go to shelf 5" }, { "from": "human",  "value": "On the shelf 5, you see nothing." }, { "from": "gpt",  "value": "ACTION: go to dresser 1" }, { "from": "human",  "value": "On the dresser 1, you see a alarmclock 2, a cd 2, a creditcard 2, a desklamp 1, a pen 2, and a pen 1." }, { "from": "gpt",  "value": "ACTION: put cellphone 1 in/on dresser 1" }, { "from": "human",  "value": "You put the cellphone 1 in/on the dresser 1." }, { "from": "gpt",  "value": "THOUGHT: I still need another cellphone to complete the task. The desk and the shelf could be likely places where I can find one.\n ACTION: go to desk 1" }, { "from": "human",  "value": "On the desk 1, you see a book 1, a bowl 1, a keychain 1, a laptop 1, and a pencil 2." }, { "from": "gpt",  "value": "ACTION: go to shelf 2" }, { "from": "human",  "value": "On the shelf 2, you see a alarmclock 1, and a creditcard 1." }, { "from": "gpt",  "value": "ACTION: go to shelf 3" }, { "from": "human",  "value": "On the shelf 3, you see a pencil 1." }, { "from": "gpt",  "value": "ACTION: go to shelf 4" }, { "from": "human",  "value": "On the shelf 4, you see a cd 1." }, { "from": "gpt",  "value": "ACTION: go to shelf 5" }, { "from": "human",  "value": "On the shelf 5, you see nothing." }, { "from": "gpt",  "value": "ACTION: go to bed 1" }, { "from": "human",  "value": "On the bed 1, you see a book 2, a cellphone 3, a cellphone 2, and a pillow 1." }, { "from": "gpt",  "value": "ACTION: take cellphone 3 from bed 1" }, { "from": "human",  "value": "You pick up the cellphone 3 from the bed 1." }, { "from": "gpt",  "value": "ACTION: go to dresser 1" }, { "from": "human",  "value": "On the dresser 1, you see a alarmclock 2, a cd 2, a cellphone 1, a creditcard 2, a desklamp 1, a pen 2, and a pen 1." }, { "from": "gpt",  "value": "ACTION: put cellphone 3 in/on dresser 1" } ]

In [22]:
one_shot_messages = []
for dialog in example_1:
    if dialog['from'] == 'human':
        msg = HumanMessage(content=dialog['value'])
    elif dialog['from'] == 'gpt':
        msg = AIMessage(content=dialog['value'])
    one_shot_messages.append(msg)
    

In [34]:
example_1[3]

{'from': 'gpt',
 'value': "THOUGHT: First, I need to find the cellphones. I'm going to start by looking in the most likely locations, such as the desk and the drawers.\n ACTION: go to desk 1"}

### Householding Environment

In [75]:
environment_prompt_template = '''Your Goal is to simulate a household environment that an agent is acting in. You will provide the output from the environment.  Below is the task the agent has been given. 

Task: 

{}

--------
For each turn, you will receive an action chosen by the agent. You must then respond as the environment listing any available actions that the agent may take next. If the user does not select an action from the provided actions, you must reply "Nothing happened." You should strive to provide actions that are realistic to the environment. You should always remind the agent of any items they posess. If the user has accomplished the task, YOU MUST call the success function: "success('[why task was successful]')". You should make sure to provide actions that relate to any items the agent possess and permit the agent to complete the goal. YOU MUST PROVIDE ACTIONS THAT ARE CONSISTENT WITH YOUR PAST ACTIONS. Please acknowledge that you understand and let's begin.

'''

hh_agent_prompt =  "Interact with a household to solve a task. Imagine you are an intelligent agent in a household environment and your target is to perform actions to complete the task goal. At the beginning of your interactions, you will be given the detailed description of the current environment and your goal to accomplish. For each of your turn, you will be given a list of actions which you can choose one to perform in this turn. You should choose from two actions: \"THOUGHT\" or \"ACTION\". If you choose \"THOUGHT\", you should first think about the current condition and plan for your future actions, and then output your action in this turn. Your output must strictly follow this format:\"THOUGHT: your thoughts.\n ACTION: your next action\n\"; If you choose \"ACTION\", you should directly output the action in this turn. Your output must strictly follow this format:\"ACTION: your next action\n\". After your each turn, the environment will give you immediate feedback based on which you plan your next few steps. if the envrionment output \"Nothing happened\", that means the previous action is invalid and you should try more options.\n Reminder: \n1. the action must be chosen from the given available actions. Any actions except provided available actions will be regarded as illegal. \n2. Think when necessary, try to act directly more in the process."

hh_agent_response = "OK. I'll follow your instructions and try my best to solve the task."



In [6]:
task = "Here is your task. You are in the middle of a room. Looking quickly around you, you see a armchair 2, a armchair 1, a bed 1, a countertop 1, a diningtable 1, a drawer 2, a drawer 1, a dresser 1, a garbagecan 1, a laundryhamper 1, and a sidetable 1.\nYour task is to: find two laptop and put them in bed."


In [24]:
agent_messages = one_shot_messages + [
    HumanMessage(content=hh_agent_prompt),
    AIMessage(content=hh_agent_response),
    HumanMessage(content=task)
]




In [25]:
agent_messages

[HumanMessage(content='Interact with a household to solve a task. Imagine you are an intelligent agent in a household environment and your target is to perform actions to complete the task goal. At the beginning of your interactions, you will be given the detailed description of the current environment and your goal to accomplish. For each of your turn, you will be given a list of actions which you can choose one to perform in this turn. You should choose from two actions: "THOUGHT" or "ACTION". If you choose "THOUGHT", you should first think about the current condition and plan for your future actions, and then output your action in this turn. Your output must strictly follow this format:"THOUGHT: your thoughts.\n ACTION: your next action\n"; If you choose "ACTION", you should directly output the action in this turn. Your output must strictly follow this format:"ACTION: your next action\n". After your each turn, the environment will give you immediate feedback based on which you plan 

In [26]:
agent_response = model.predict_messages(agent_messages)

In [27]:
agent_response.content

"THOUGHT: To find the laptops, I should first check places where they're commonly kept - like the desk or drawer.\n ACTION: go to drawer 1"

In [ ]:
# Create one shot prompt

In [46]:

one_shot_environment_prompt = environment_prompt_template.format(example_1[2]['value'])

one_shot_environment_messages = [
    HumanMessage(content=one_shot_environment_prompt),
    AIMessage(content="OK. I'll follow your instructions and try my best to simulate a household environment."),
]

for dialog in example_1[3:]:
    if dialog['from'] == 'human':
        msg = AIMessage(content=dialog['value'])
    elif dialog['from'] == 'gpt':
        msg = HumanMessage(content=dialog['value'])
    one_shot_environment_messages.append(msg)

one_shot_environment_messages.append(AIMessage(content="success('cellphone 1 and cellphone 3 were placed on dresser 1')"))

In [48]:
environment_prompt = environment_prompt_template.format(task)

environment_messages = one_shot_environment_messages + [
    HumanMessage(content=environment_prompt),
    AIMessage(content="OK. I'll follow your instructions and try my best to simulate a household environment."),
    HumanMessage(content=agent_response.content)
]

In [49]:
environment_result = model.predict_messages(environment_messages)

In [51]:
environment_result.content

'The drawer 1 is closed.'

In [52]:
environment_messages

[HumanMessage(content='Your Goal is to simulate a household environment that an agent is acting in. You will provide the output from the environment.  Below is the task the agent has been given. \n\nTask: \n\nHere is your task. You are in the middle of a room. Looking quickly around you, you see a bed 1, a desk 1, a drawer 8, a drawer 7, a drawer 6, a drawer 5, a drawer 4, a drawer 3, a drawer 2, a drawer 1, a dresser 1, a garbagecan 1, a shelf 5, a shelf 4, a shelf 3, a shelf 2, and a shelf 1.\nYour task is to: put two cellphone in dresser.\n\n--------\nFor each turn, you will receive an action chosen by the agent. You must then respond as the environment listing any available actions that the agent may take next. If the user does not select an action from the provided actions, you must reply "Nothing happened." You should strive to provide actions that are realistic to the environment. You should always remind the agent of any items they posess. You should make sure to provide action

In [53]:
agent_messages.append(HumanMessage(content=environment_result.content))

In [55]:
num_turns = 10

for i in range(num_turns):
    agent_response = model.predict_messages(agent_messages)
    print(agent_response.content)
    agent_result = agent_response.content
    environment_messages.append(HumanMessage(content=agent_result))
    agent_messages.append(agent_response)
    environment_result = model.predict_messages(environment_messages)
    if "success(" in environment_result.content:
        break
    environment_messages.append(environment_result)
    print(environment_result.content)
    agent_messages.append(HumanMessage(content=environment_result.content))

ACTION: open drawer 1
You open the drawer 1. The drawer 1 is open. In it, you see a keychain 1, a laptop 1, and a notebook 1.
ACTION: take laptop 1 from drawer 1
You pick up the laptop 1 from the drawer 1. Now you have a laptop 1.
ACTION: go to drawer 2
The drawer 2 is closed.
ACTION: open drawer 2
You open the drawer 2. The drawer 2 is open. In it, you see a chargingcable 1 and a pencil 1.
ACTION: go to dresser 1
The dresser 1 is closed.
ACTION: open dresser 1
You open the dresser 1. The dresser 1 is open. In it, you see a book 1, a laptop 2, and a sock 1.
ACTION: take laptop 2 from dresser 1
You pick up the laptop 2 from the dresser 1. Now you have a laptop 1 and a laptop 2.
ACTION: go to bed 1
On the bed 1, you see a blanket 1 and a pillow 1.
ACTION: put laptop 1 in/on bed 1
You put the laptop 1 in/on the bed 1. Now you have a laptop 2.
ACTION: put laptop 2 in/on bed 1


#### Get tasks from AgentInstruct Data

In [57]:
import pandas as pd
df = pd.read_parquet('~/spikes/notebook_spikes/agentinstruct/alfworld-00000-of-00001-302ad687bb3817a4.parquet')
print(df)

                                         conversations            id
0    [{'from': 'human', 'loss': None, 'value': 'Int...    alfworld_0
1    [{'from': 'human', 'loss': None, 'value': 'Int...    alfworld_1
2    [{'from': 'human', 'loss': None, 'value': 'Int...    alfworld_2
3    [{'from': 'human', 'loss': None, 'value': 'Int...    alfworld_3
4    [{'from': 'human', 'loss': None, 'value': 'Int...    alfworld_4
..                                                 ...           ...
331  [{'from': 'human', 'loss': None, 'value': 'Int...  alfworld_331
332  [{'from': 'human', 'loss': None, 'value': 'Int...  alfworld_332
333  [{'from': 'human', 'loss': None, 'value': 'Int...  alfworld_333
334  [{'from': 'human', 'loss': None, 'value': 'Int...  alfworld_334
335  [{'from': 'human', 'loss': None, 'value': 'Int...  alfworld_335

[336 rows x 2 columns]


In [58]:
print(df.iloc[2]["conversations"][2]['value'])

Here is your task. You are in the middle of a room. Looking quickly around you, you see a cabinet 11, a cabinet 10, a cabinet 9, a cabinet 8, a cabinet 7, a cabinet 6, a cabinet 5, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a coffeemachine 1, a countertop 2, a countertop 1, a diningtable 2, a diningtable 1, a drawer 3, a drawer 2, a drawer 1, a fridge 1, a garbagecan 1, a microwave 1, a sidetable 1, a sinkbasin 1, a stoveburner 4, a stoveburner 3, a stoveburner 2, a stoveburner 1, and a toaster 1.
Your task is to: heat some plate and put it in countertop.


In [59]:
def process(data):
    return (data['conversations'][2]['value'])

In [60]:
task_and_environment = df.apply(process, axis=1).tolist()

In [61]:
len(task_and_environment )

336

In [66]:
task_and_environment[305]

'Here is your task. You are in the middle of a room. Looking quickly around you, you see a cabinet 27, a cabinet 26, a cabinet 25, a cabinet 24, a cabinet 23, a cabinet 22, a cabinet 21, a cabinet 20, a cabinet 19, a cabinet 18, a cabinet 17, a cabinet 16, a cabinet 15, a cabinet 14, a cabinet 13, a cabinet 12, a cabinet 11, a cabinet 10, a cabinet 9, a cabinet 8, a cabinet 7, a cabinet 6, a cabinet 5, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a coffeemachine 1, a countertop 2, a countertop 1, a diningtable 1, a drawer 12, a drawer 11, a drawer 10, a drawer 9, a drawer 8, a drawer 7, a drawer 6, a drawer 5, a drawer 4, a drawer 3, a drawer 2, a drawer 1, a fridge 1, a garbagecan 1, a microwave 1, a sinkbasin 1, a stoveburner 4, a stoveburner 3, a stoveburner 2, a stoveburner 1, and a toaster 1.\nYour task is to: heat some mug and put it in coffeemachine.'

### Create Training Data Conversations

In [76]:
def one_shot_environment_setup():
    one_shot_environment_prompt = environment_prompt_template.format(example_1[2]['value'])
    one_shot_environment_messages = [
        HumanMessage(content=one_shot_environment_prompt),
        AIMessage(content="OK. I'll follow your instructions and try my best to simulate a household environment."),
    ]

    for dialog in example_1[3:]:
        if dialog['from'] == 'human':
            msg = AIMessage(content=dialog['value'])
        elif dialog['from'] == 'gpt':
            msg = HumanMessage(content=dialog['value'])
        one_shot_environment_messages.append(msg)

    one_shot_environment_messages.append(AIMessage(content="success('cellphone 1 and cellphone 3 were placed on dresser 1')"))
    return one_shot_environment_messages

In [77]:
num_turns = 40

all_agents = []
all_environments = []
start = 0

for x in range(start,start+len(task_and_environment)): 
    task = task_and_environment[x]
    environment_info = task_and_environment[x][1]
    print("Task {}:".format(x), task)
    agent_messages = one_shot_messages + [
        HumanMessage(content=hh_agent_prompt),
        AIMessage(content=hh_agent_response),
        HumanMessage(content=task)
    ]

    agent_response = model.predict_messages(agent_messages)
    print(agent_response.content)
    agent_messages.append(agent_response)
    
    environment_message = one_shot_environment_setup()
    environment_prompt = environment_prompt_template.format(task)
    environment_messages = one_shot_environment_messages + [
        HumanMessage(content=environment_prompt),
        AIMessage(content="OK. I'll follow your instructions and try my best to simulate a household environment."),
        HumanMessage(content=agent_response.content)
    ]
    environment_result = model.predict_messages(environment_messages)
    environment_messages.append(environment_result)
    agent_messages.append(HumanMessage(content=environment_result.content))
    print(environment_result.content)
    for i in range(num_turns):
        agent_response = model.predict_messages(agent_messages)
        print(agent_response.content)
        environment_messages.append(HumanMessage(content=agent_response.content))
        agent_messages.append(agent_response)
        environment_result = model.predict_messages(environment_messages)
        environment_messages.append(environment_result)
        print(environment_result.content)
        agent_messages.append(HumanMessage(content=environment_result.content))
        if "success(" in environment_result.content:
            break
    all_agents.append(deepcopy(agent_messages))
    all_environments.append(deepcopy(environment_messages))
    print("\n\n")

Task 0: Here is your task. You are in the middle of a room. Looking quickly around you, you see a armchair 2, a armchair 1, a bed 1, a countertop 1, a diningtable 1, a drawer 2, a drawer 1, a dresser 1, a garbagecan 1, a laundryhamper 1, and a sidetable 1.
Your task is to: find two laptop and put them in bed.
THOUGHT: The laptops could be on the dining table, countertop, or inside the drawers. I should start looking there.
 ACTION: go to diningtable 1
On the diningtable 1, you see a bowl 1, a fork 1, a knife 1, a plate 1, and a spoon 1.
ACTION: go to countertop 1
On the countertop 1, you see a blender 1, laptop 1, a coffee maker 1, a cutting board 1, a knife 2, and a toaster 1.
ACTION: take laptop 1 from countertop 1
You pick up the laptop 1 from the countertop 1.
ACTION: go to drawer 1
The drawer 1 is closed.
ACTION: open drawer 1
You open the drawer 1. The drawer 1 is open. In it, you see nothing.
ACTION: go to drawer 2
The drawer 2 is closed.
ACTION: open drawer 2
You open the drawe

ACTION: take pencil 1 from drawer 1
You pick up the pencil 1 from the drawer 1.
ACTION: go to desk 1
On the desk 1, you see a keychain 1, a laptop 1, and a book 1.
ACTION: put pencil 1 in/on desk 1
success('pencil 1 was placed on desk 1')



Task 5: Here is your task. You are in the middle of a room. Looking quickly around you, you see a cabinet 13, a cabinet 12, a cabinet 11, a cabinet 10, a cabinet 9, a cabinet 8, a cabinet 7, a cabinet 6, a cabinet 5, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a coffeemachine 1, a countertop 1, a diningtable 1, a drawer 1, a fridge 1, a garbagecan 1, a microwave 1, a shelf 3, a shelf 2, a shelf 1, a sinkbasin 1, a stoveburner 4, a stoveburner 3, a stoveburner 2, a stoveburner 1, and a toaster 1.
Your task is to: put a cool plate in shelf.
THOUGHT: The first step is to locate a cool plate. It might be in the cabinets or on the countertop.
 ACTION: go to cabinet 1
The cabinet 1 is closed.
ACTION: open cabinet 1
You open the cabinet 1. The cab

The drawer 1 is closed.
ACTION: open drawer 1
You open drawer 1. In it, you see a ruler 1, and a pair of scissors 1.
ACTION: go to drawer 2
The drawer 2 is closed.
ACTION: open drawer 2
You open drawer 2. In it, you see a pen 1, a stapler 1, and a paperclip 1.
ACTION: take pen 1 from drawer 2
You pick up the pen 1 from the drawer 2.
ACTION: go to sidetable 1
On the sidetable 1, you see a lamp 1, a book 1, and a glasses 1.
ACTION: put pen 1 on sidetable 1
success('pen 1 was placed on sidetable 1')



Task 11: Here is your task. You are in the middle of a room. Looking quickly around you, you see a cabinet 13, a cabinet 12, a cabinet 11, a cabinet 10, a cabinet 9, a cabinet 8, a cabinet 7, a cabinet 6, a cabinet 5, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a coffeemachine 1, a countertop 2, a countertop 1, a diningtable 1, a drawer 4, a drawer 3, a drawer 2, a drawer 1, a fridge 1, a garbagecan 1, a microwave 1, a shelf 3, a shelf 2, a shelf 1, a sinkbasin 1, a stoveburner 4, a

You pick up the spraybottle 1 from the cabinet 1.
ACTION: go to toilet 1
The toilet 1 is closed.
ACTION: put spraybottle 1 on toilet 1
success('spraybottle 1 was placed on toilet 1')



Task 17: Here is your task. You are in the middle of a room. Looking quickly around you, you see a armchair 3, a armchair 2, a armchair 1, a bed 1, a cabinet 2, a cabinet 1, a desk 1, a drawer 4, a drawer 3, a drawer 2, a drawer 1, a garbagecan 1, a laundryhamper 1, a safe 1, and a sidetable 1.
Your task is to: look at pen under the desklamp.
THOUGHT: To look at pen under the desklamp, I first need to find the desklamp. It could be on the desk or the sidetable.
 ACTION: go to desk 1
On the desk 1, you see a book 1, a bowl 1, a keychain 1, a laptop 1, a pencil 2, and desklamp 1.
THOUGHT: The desklamp is on the desk, but there's no pen under it here. However, the pen might be hidden, and I haven't interacted with the desk lamp yet.
 ACTION: look under desklamp 1
You look under the desklamp 1. There, you i

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


...

THOUGHT: Now that I have the dishsponge, I should clean it. The sinkbasin is a likely place to do this. 
 ACTION: go to sinkbasin 1
On the sinkbasin 1, you see a faucet 1.
ACTION: clean dishsponge 1 with faucet 1
You clean the dishsponge 1 with the faucet 1. The dishsponge 1 is now clean.
THOUGHT: Now that the dishsponge is clean, I should put it in the shelf as instructed. 
 ACTION: go to shelf 1
On the shelf 1, you see a cup 1, a plate 1, and a bowl 1.
ACTION: put dishsponge 1 on shelf 1
success('The clean dishsponge 1 was placed on shelf 1')



Task 316: Here is your task. You are in the middle of a room. Looking quickly around you, you see a armchair 1, a bed 1, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a drawer 6, a drawer 5, a drawer 4, a drawer 3, a drawer 2, a drawer 1, a dresser 1, a garbagecan 1, a sidetable 2, and a sidetable 1.
Your task is to: put some pen on sidetable.
THOUGHT: The best places to find a pen would be the desk, drawers or cabinets. I'll start with

ACTION: put cool mug 1 in/on coffeemachine 1
success('cool mug 1 was placed in/on coffeemachine 1')



Task 321: Here is your task. You are in the middle of a room. Looking quickly around you, you see a armchair 3, a armchair 2, a armchair 1, a bed 1, a cabinet 2, a cabinet 1, a desk 1, a drawer 4, a drawer 3, a drawer 2, a drawer 1, a garbagecan 1, a laundryhamper 1, a safe 1, and a sidetable 1.
Your task is to: find two book and put them in desk.
THOUGHT: The most probable places to find books are the drawers, the cabinet and the sidetable. I will start by looking in the drawers.
 ACTION: go to drawer 1
The drawer 1 is closed.
ACTION: open drawer 1
You open the drawer 1. The drawer 1 is open. In it, you see a book 1 and a key 1.
ACTION: take book 1 from drawer 1
You pick up the book 1 from the drawer 1.
ACTION: go to drawer 2
The drawer 2 is closed.
ACTION: open drawer 2
You open the drawer 2. The drawer 2 is open. In it, you see nothing.
ACTION: go to drawer 3
The drawer 3 is closed

THOUGHT: Now I have two cellphones. I need to put them in the shelf as per the task.
 ACTION: go to shelf 1
On the shelf 1, you see a book 4, a keychain 3, and a lamp 1.
ACTION: put cellphone 1 in/on shelf 1
You put the cellphone 1 in/on the shelf 1. Now you have a cellphone 2.
ACTION: put cellphone 2 in/on shelf 1
You put the cellphone 2 in/on the shelf 1. Now you have no items. 
success('cellphone 1 and cellphone 2 were placed on shelf 1')



Task 326: Here is your task. You are in the middle of a room. Looking quickly around you, you see a bed 1, a desk 1, a drawer 5, a drawer 4, a drawer 3, a drawer 2, a drawer 1, a dresser 1, and a garbagecan 1.
Your task is to: put some alarmclock on desk.
THOUGHT: First, I need to find an alarm clock. I'm going to start by looking in the most likely locations, such as the dresser, drawers, and if not found, I will check the bed.
 ACTION: go to dresser 1
On the dresser 1, you see a book 1, a bowl 1, an alarmclock 1, a jewelrybox 1, and a picturef

Nothing happened.
THOUGHT: I have accomplished the task. There are no further actions required.
 ACTION: do nothing
Nothing happened.
THOUGHT: I've completed the task. There's no need for further actions.
 ACTION: do nothing
Nothing happened.
THOUGHT: I've accomplished the task. No further actions are required.
 ACTION: do nothing
Nothing happened.
THOUGHT: I've completed the task. No further actions are required.
 ACTION: do nothing
Nothing happened.
THOUGHT: I've accomplished the task. There's nothing else to do.
 ACTION: do nothing
Nothing happened.
THOUGHT: The task has been completed. There's nothing else I need to do.
 ACTION: do nothing
Nothing happened.
THOUGHT: The task has been completed. There's nothing else I need to do.
 ACTION: do nothing
Nothing happened.
THOUGHT: The task is completed. There's nothing else required at this moment.
 ACTION: do nothing
Nothing happened.
THOUGHT: The task has been completed. There's nothing else I need to do.
 ACTION: do nothing
Nothing ha

THOUGHT: I am not sure where the cd is, so I'll need to search the room. The drawers seem like a likely place to find a cd. I'm going to start with the first drawer.
 ACTION: go to drawer 1
The drawer 1 is closed.
ACTION: open drawer 1
You open drawer 1. Inside, you see a flashlight 1.
ACTION: go to drawer 2
The drawer 2 is closed.
ACTION: open drawer 2
You open drawer 2. Inside, you see nothing.
ACTION: go to drawer 3
The drawer 3 is closed.
ACTION: open drawer 3
You open drawer 3. Inside, you see a cellphone 1.
ACTION: go to drawer 4
The drawer 4 is closed.
ACTION: open drawer 4
You open drawer 4. Inside, you see a cd 1.
ACTION: take cd 1 from drawer 4
You pick up the cd 1 from the drawer 4.
ACTION: go to sidetable 1
The sidetable 1 is closed.
ACTION: open sidetable 1
You open sidetable 1. Inside, you see nothing.
ACTION: put cd 1 in sidetable 1
success('cd 1 was placed in sidetable 1')





In [71]:
len(all_environments)

2

In [78]:
conversations_ = []
for chat in all_agents:
    conversation_ = []
    for side in chat:
        if side.type == "human":
            side_type = "human"
        else:
            side_type = "gpt"
        conversation_.append({"from": side_type, "value":side.content})
    conversations_.append(conversation_ )
        

In [79]:
with open('./synthetic/householding-interactions-raw.jsonl', 'w') as file:
    for conversation in conversations_:
        json_str = json.dumps(conversation)
        file.write(json_str + "\n")